# Visualization of results for Logistic Regression

@author: Caroline Gasten

The present notebook visualizes odds ratios, as obtained from the Logistic Regression, as tree diagrams

## Settings

In [ ]:
#required packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import os
import numpy as np

In [ ]:
#paths
path_input = #path to results from logistic regression
path_figs = #path to figures

In [ ]:
#Figure style
tw_pt = 483.69
tw_inch = tw_pt/72.27
mpl.rc('figure', titlesize=11, labelsize=10, figsize=[tw_inch,tw_inch*1.4])#https://jwalton.info/Embed-Publication-Matplotlib-Latex/
mpl.rc('axes', titlesize=11, labelsize=10)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)
mpl.rc('legend', fontsize=8)
colour_rng=['teal', 'salmon', 'lightblue', 'silver', 'darkseagreen', 'palevioletred', 'slategray']
mpl.rcParams['axes.prop_cycle'] = cycler(color=colour_rng)

## Tree diagrams

In [ ]:
def confint_intervals(model, focus):
    """
    The present function creates a tree diagram for the results of a model for positive or negative drought indicator across all aggregation periods and all temporal lags 
    """
    #dictionaries with keywords
    components_dict = {'Model1':[''], 'Model2':['', '_xturkana', '_xwestpokot'], 'Model3':[''], 'Model4':['','_xTU','_xPO', '_xTO', '_xGA','_xDA']}
    legend_dict = {'Model1':['confidence interval'], 'Model2':['Marsabit', 'Turkana', 'West Pokot'], 'Model3':['confidence interval'], 'Model4':['Borana','Turkana','Pokot', 'Toposa', 'Gabra','Dassanetch']}
    focus_dict ={'_dry':'negative', '_wet':'positive'}
    
    #retrieve keywords for model at hand
    components = components_dict[model]
    legend_items = legend_dict[model]
    
    #SPI
    f, axes = plt.subplots(4,1, sharex=True, sharey=True)
    DIs = ['SPI-1',  'SPI-3',  'SPI-6', 'SPI-12']
    
    #loop through different cross-sectional individuals
    for c in range(len(components)):
        #load results and retrieve odds ratios
        df = pd.read_csv(os.path.join(path_input, model, 'R_%s%s%s_coeffs2years.csv'%(model, focus, components[c])), header=[0,1]).rename(columns={"Unnamed: 1_level_1":"stats", "Unnamed: 0_level_0":"0", "Unnamed: 1_level_0":"1"}).set_index([("0", "time_lag [months]"), ("1", "stats")])
        df_ss = df.iloc[df.index.get_level_values(1)=='Median', df.columns.get_level_values(0).isin(['odds-factor', 'oddsconfint_025', 'oddsconfint_05', 'oddsconfint_95', 'oddsconfint_975'])].stack(level=0)
        for i in range(4):
            ax=axes.flatten()[i]
            
            #plot 95% confidence interval of odds ratios as line
            ax.vlines(np.arange(8)+c*0.1, ymin=df_ss.loc[(DIs[i], 'Median', 'oddsconfint_975')], ymax=df_ss.loc[(DIs[i], 'Median', 'oddsconfint_025')], lw=2, color=colour_rng[c], label=legend_items[c])
            #plot odds ratio expected value as point
            if c<len(components)-1:
                ax.plot(np.arange(8)+c*0.1, df_ss.loc[(DIs[i], 'Median', 'odds-factor')],  'o', ms=4, markeredgecolor=colour_rng[c], markerfacecolor=colour_rng[c])
            else:
                ax.plot(np.arange(8)+c*0.1, df_ss.loc[(DIs[i], 'Median', 'odds-factor')],  'o', ms=4, markeredgecolor=colour_rng[c], markerfacecolor=colour_rng[c], label='expected value of OR')
            #horizontal line at OR=1
            ax.axhline(y=1, ls='--', lw=0.4, color='gray')
            
            ax.set_ylim(0,3.5)
            ax.set_title(DIs[i])
            #ax.set_xlabel('lag')
    for ax in axes:
        #ylabel creation
        if focus=='_dry':
            label_x1 = -0.12
            label_x2 = -0.09

            ax.text(label_x1, 0.05, r"OR for a unit decrease", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.17, r"in", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.26, r"negative SPI", color='red', weight='bold',  rotation='vertical', transform=ax.transAxes)
        else:
            label_x1 = -0.12
            label_x2 = -0.09

            ax.text(label_x1, 0.05, r"OR for a unit increase", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.17, r"in", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.26, r"positive SPI", color='blue', weight='bold',  rotation='vertical', transform=ax.transAxes)
        
    #formatting
    handles, labels = ax.get_legend_handles_labels()
    plt.xlabel('time lag [months]')
    f.legend(handles=handles, loc=(0.15, 0.92), ncols=4)
    plt.suptitle("Model %s: 95%% confidence intervals of odds ratio (OR) over %s SPI"%(model[-1], focus_dict[focus]))
    plt.tight_layout(rect = [0,0, 1, 0.95])
    plt.savefig(os.path.join(path_figs, '%s%s_SPI_confint.svg'%(model, focus)))
    
    
    #SPEI
    f, axes = plt.subplots(4,1, sharex=True, sharey=True)
    DIs = ['SPEI-1',  'SPEI-3',  'SPEI-6', 'SPEI-12']
    
    #loop through different cross-sectional individuals
    for c in range(len(components)):
        #load results and retrieve odds ratios
        df = pd.read_csv(os.path.join(path_input, model, 'R_%s%s%s_coeffs2years.csv'%(model, focus, components[c])), header=[0,1]).rename(columns={"Unnamed: 1_level_1":"stats", "Unnamed: 0_level_0":"0", "Unnamed: 1_level_0":"1"}).set_index([("0", "time_lag [months]"), ("1", "stats")])
        df_ss = df.iloc[df.index.get_level_values(1)=='Median', df.columns.get_level_values(0).isin(['odds-factor', 'oddsconfint_025', 'oddsconfint_05', 'oddsconfint_95', 'oddsconfint_975'])].stack(level=0)
        for i in range(4):
            ax=axes.flatten()[i]
            #plot 95% confidence interval of odds ratios as line
            ax.vlines(np.arange(8)+c*0.1, ymin=df_ss.loc[(DIs[i], 'Median', 'oddsconfint_975')], ymax=df_ss.loc[(DIs[i], 'Median', 'oddsconfint_025')], lw=2, color=colour_rng[c], label=legend_items[c])
            #plot odds ratio expected value as point
            if c<len(components)-1:
                ax.plot(np.arange(8)+c*0.1, df_ss.loc[(DIs[i], 'Median', 'odds-factor')],  'o', ms=4, markeredgecolor=colour_rng[c], markerfacecolor=colour_rng[c])
            else:
                ax.plot(np.arange(8)+c*0.1, df_ss.loc[(DIs[i], 'Median', 'odds-factor')],  'o', ms=4, markeredgecolor=colour_rng[c], markerfacecolor=colour_rng[c], label='expected value of OR')
            #horizontal line at OR=1
            ax.axhline(y=1, ls='--', lw=0.4, color='gray')
            ax.set_ylim(0,3.5)
            ax.set_title(DIs[i])
            #ax.set_xlabel('lag')
    for ax in axes:
        #ylabel creation
        if focus=='_dry':
            label_x1 = -0.12
            label_x2 = -0.09

            ax.text(label_x1, 0.05, r"OR for a unit decrease", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.17, r"in", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.26, r"negative SPEI", color='red', weight='bold',  rotation='vertical', transform=ax.transAxes)
        else:
            label_x1 = -0.12
            label_x2 = -0.09

            ax.text(label_x1, 0.05, r"OR for a unit increase", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.17, r"in", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.26, r"positive SPEI", color='blue', weight='bold',  rotation='vertical', transform=ax.transAxes)
        
    #formatting
    handles, labels = ax.get_legend_handles_labels()
    plt.xlabel('time lag [months]')
    f.legend(handles=handles, loc=(0.15, 0.92), ncols=4)
    plt.suptitle("Model %s: 95%% confidence intervals of odds ratio (OR) over %s SPEI"%(model[-1], focus_dict[focus]))
    plt.tight_layout(rect = [0,0, 1, 0.95])
    plt.savefig(os.path.join(path_figs, '%s%s_SPEI_confint.svg'%(model, focus)))

In [ ]:
#loop through all models and negative and positive DIs to create tree diagrams
for model in ['Model1', 'Model2', 'Model3', 'Model4']:
    for focus in ['_dry', '_wet']:
        confint_intervals(model, focus)

In [ ]:
#subset of function above to create diagram for SPEI-6 across all four models

Models = ['Model1','Model2', 'Model3', 'Model4']
DI = 'SPEI-6'
focus = '_dry'

components_dict = {'Model1':[''], 'Model2':['', '_xturkana', '_xwestpokot'], 'Model3':[''], 'Model4':['','_xTU','_xPO', '_xTO', '_xGA','_xDA']}
legend_dict = {'Model1':['_'], 'Model2':['Marsabit', 'Turkana', 'West Pokot'], 'Model3':['_'], 'Model4':['Borana','Turkana','Pokot', 'Toposa', 'Gabra','Dassanetch']}
focus_dict ={'_dry':'negative', '_wet':'positive'}

f, axes = plt.subplots(4,1, sharex=True, sharey=True)


for i in range(4):
    model=Models[i]
    components = components_dict[model]
    legend_items = legend_dict[model]
    
    ax=axes[i]
    
    for c in range(len(components)):
        df = pd.read_csv(os.path.join(path_input, model, 'R_%s%s%s_coeffs2years.csv'%(model, focus, components[c])), header=[0,1]).rename(columns={"Unnamed: 1_level_1":"stats", "Unnamed: 0_level_0":"0", "Unnamed: 1_level_0":"1"}).set_index([("0", "time_lag [months]"), ("1", "stats")])
        df_ss = df.iloc[df.index==(DI,'Median'), df.columns.get_level_values(0).isin(['odds-factor', 'oddsconfint_025', 'oddsconfint_05', 'oddsconfint_95', 'oddsconfint_975'])].stack(level=0)
        
        ax.vlines(np.arange(8)+c*0.1, ymin=df_ss.loc[(DI, 'Median', 'oddsconfint_975')], ymax=df_ss.loc[(DI, 'Median', 'oddsconfint_025')], lw=2, color=colour_rng[c], label=legend_items[c])
        
    
        ax.plot(np.arange(8)+c*0.1, df_ss.loc[(DI, 'Median', 'odds-factor')],  'o', ms=4, markeredgecolor=colour_rng[c], markerfacecolor=colour_rng[c])
        ax.axhline(y=1, ls='--', lw=0.4, color='gray')
        ax.set_ylim(0,3.5)
        #ax.set_xlabel('lag')
    if focus=='_dry':
        label_x1 = -0.12
        label_x2 = -0.09

        ax.text(label_x1, 0.05, r"OR for a unit decrease", rotation='vertical', transform=ax.transAxes)
        ax.text(label_x2, 0.17, r"in", rotation='vertical', transform=ax.transAxes)
        ax.text(label_x2, 0.26, r"negative SPEI",  rotation='vertical', transform=ax.transAxes)
    else:
        label_x1 = -0.12
        label_x2 = -0.09

        ax.text(label_x1, 0.05, r"OR for a unit increase", rotation='vertical', transform=ax.transAxes)
        ax.text(label_x2, 0.17, r"in", rotation='vertical', transform=ax.transAxes)
        ax.text(label_x2, 0.26, r"positive SPEI", color='blue',  rotation='vertical', transform=ax.transAxes)
    ax.set_title('Model ' + model[-1])
    if model == 'Model2':
        ax.legend(loc='upper right', ncols=len(components))
    elif model =='Model4':
        ax.legend(loc='upper right', ncols=3)
    
    plt.xlabel('time lag [months]')
    plt.suptitle('Expected value and 95% confidence interval of the odds ratio (OR)\nfor a unit decrease in negative SPEI-6 for all four models')
    plt.tight_layout()
    plt.savefig(os.path.join(path_figs, 'ComparisonModelsSPEI6.svg'))

In [ ]:
#subset of function above to create diagrams for negative and positive SPEI-1 and SPEI-6 for Model 4
model='Model4'
for focus in ['_wet', '_dry']:

    components_dict = {'Model1':[''], 'Model2':['', '_xturkana', '_xwestpokot'], 'Model3':[''], 'Model4':['','_xTU','_xPO', '_xTO', '_xGA','_xDA']}
    legend_dict = {'Model1':['confidence interval'], 'Model2':['Marsabit', 'Turkana', 'West Pokot'], 'Model3':['confidence interval'], 'Model4':['Borana','Turkana','Pokot', 'Toposa', 'Gabra','Dassanetch']}
    focus_dict ={'_dry':'negative', '_wet':'positive'}
    components = components_dict[model]
    legend_items = legend_dict[model]

    #SPEI
    f, axes = plt.subplots(2,1, sharex=True, sharey=True,figsize=[tw_inch,tw_inch*0.7])
    DIs = ['SPEI-1', 'SPEI-6']
    for c in range(len(components)):
        df = pd.read_csv(os.path.join(path_input, model, 'R_%s%s%s_coeffs2years.csv'%(model, focus, components[c])), header=[0,1]).rename(columns={"Unnamed: 1_level_1":"stats", "Unnamed: 0_level_0":"0", "Unnamed: 1_level_0":"1"}).set_index([("0", "time_lag [months]"), ("1", "stats")])
        df_ss = df.iloc[df.index.get_level_values(1)=='Median', df.columns.get_level_values(0).isin(['odds-factor', 'oddsconfint_025', 'oddsconfint_05', 'oddsconfint_95', 'oddsconfint_975'])].stack(level=0)
        for i in range(2):
            ax=axes.flatten()[i]
            ax.vlines(np.arange(8)+c*0.1, ymin=df_ss.loc[(DIs[i], 'Median', 'oddsconfint_975')], ymax=df_ss.loc[(DIs[i], 'Median', 'oddsconfint_025')], lw=2, color=colour_rng[c], label=legend_items[c])
            if c<len(components)-1:
                ax.plot(np.arange(8)+c*0.1, df_ss.loc[(DIs[i], 'Median', 'odds-factor')],  'o', ms=4, markeredgecolor=colour_rng[c], markerfacecolor=colour_rng[c])
            else:
                ax.plot(np.arange(8)+c*0.1, df_ss.loc[(DIs[i], 'Median', 'odds-factor')],  'o', ms=4, markeredgecolor=colour_rng[c], markerfacecolor=colour_rng[c], label='expected value of OR')
            ax.axhline(y=1, ls='--', lw=0.4, color='gray')
            ax.set_ylim(0,3.5)
            ax.set_title(DIs[i])
            #ax.set_xlabel('lag')
    for ax in axes:
        if focus=='_dry':
            label_x1 = -0.12
            label_x2 = -0.09

            ax.text(label_x1, 0, r"OR for a unit decrease", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.12, r"in", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.23, r"negative SPEI", color='red', weight='bold',  rotation='vertical', transform=ax.transAxes)
        else:
            label_x1 = -0.12
            label_x2 = -0.09

            ax.text(label_x1, 0, r"OR for a unit increase", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.12, r"in", rotation='vertical', transform=ax.transAxes)
            ax.text(label_x2, 0.23, r"positive SPEI", color='blue', weight='bold',  rotation='vertical', transform=ax.transAxes)


    handles, labels = ax.get_legend_handles_labels()
    plt.xlabel('time lag [months]')
    f.legend(handles=handles, loc=(0.15, 0.87), ncols=4)
    plt.suptitle("Model %s: 95%% confidence intervals of odds ratio (OR) over %s SPEI-1 and SPEI-6"%(model[-1], focus_dict[focus]))
    plt.tight_layout(rect = [0,0, 1, 0.95])
    plt.savefig(os.path.join(path_figs, '%s%s_SPEI_1_6_confint.svg'%(model, focus)))